In [1]:
import network_utils as ne
import networkx as nx
import powerlaw
import numpy as np
import pandas as pd
import multiprocessing as mp

#### Function calls

In [2]:
def generate_synthetic_ks(degree_sequence, ntail, xmin) -> list[int]:
    """
    Generate a synthetic distribution for scale-free analysis 
    --------------------------
    Args:
        degree_sequence (list[int]): Array of node degrees.
        ntail (int): Number of nodes with degrees greater than k*_min.
        xmin (int): k*_min. 
    Returns:
        synthetic_seq (list[int]): Synthetic degree sequence. 
    """
    fit = powerlaw.Fit(degree_sequence, xmin=xmin, discrete=True, verbose=False)
    n = len(degree_sequence)
    synthetic_seq = []
    emperical_set = [deg for deg in degree_sequence if deg < xmin] # distribution of nodes degree below xmin 
    
    for _ in range(n):
        if np.random.rand() < (ntail / n): # sample from fitted power law
            synthetic_seq.append(fit.power_law.generate_random(1)[0])
        else: # sample from emperical distribution below k*_min
            synthetic_seq.append(np.random.choice(emperical_set))
    return synthetic_seq

In [3]:

def goodness_of_fit_multiprocess(degree_sequence, fit, num_synthetic=1000) -> float:
    """
    Test the goodness of fit of power law distribution 
    --------------------------
    Args:
        degree_sequence (list[int]): Emperical degree distribution.
        fit: powerlaw package fit of degree_sequence.
        num_synthetic (int): Number of synthetic distribution used for testing. 
    Returns:
        p_value (float): p_value for goodness of fit.  
    """
    ntail = sum(deg >= fit.xmin for deg in degree_sequence) # number of nodes with degree above xmin 
    
    # use multiprocessing for faster speed
    pool_args = [(degree_sequence, ntail, fit.xmin) for _ in range(num_synthetic)]
    with mp.get_context("fork").Pool(processes=mp.cpu_count()) as pool:
        results = [pool.apply_async(generate_synthetic_ks, args=args) for args in pool_args]
        synthetic_samples = [r.get() for r in results]
    
    D_synthetic = [] # fit each synethetic to its own power law
    for sample in synthetic_samples:
        fit_synthetic = powerlaw.Fit(sample, discrete=True, verbose=False)
        D_synthetic.append(fit_synthetic.D)
    p_value = np.sum(D_synthetic >= fit.D) / len(D_synthetic) # fraction of synthetic distributions with a worse KS D statistic 
    return p_value

#### Load Data

In [4]:
with pd.HDFStore('./data/gene_network_data.h5') as store:
    tec = store['TEC']
np_tec_abs = np.abs(tec.to_numpy(copy=True))

#### Scale-free analysis specifically for TEC network at threshold of 0.75

In [5]:
tec_al = ne.threshold_weighted_adjacency_list(np_tec_abs, 0.75)
tec_graph_75 = ne.construct_network(tec_al, "TEC_75", tec.columns)

In [6]:
n = nx.number_of_nodes(tec_graph_75)
ba_m = nx.number_of_edges(tec_graph_75) // nx.number_of_nodes(tec_graph_75) # edge to node ratio
G_barabasi_albert = nx.barabasi_albert_graph(n, ba_m) # generate Barabais Albert model for comparison

print(f"TEC Network: {n} nodes with {nx.number_of_edges(tec_graph_75)} edges")
print(f"Barabasi Albert Network: {nx.number_of_nodes(G_barabasi_albert)} nodes with {nx.number_of_edges(G_barabasi_albert)} edges")

TEC Network: 3804 nodes with 26878 edges
Barabasi Albert Network: 3804 nodes with 26579 edges


In [7]:
tec_degrees = [tec_graph_75.degree(n) for n in tec_graph_75.nodes()]
tec_fit = powerlaw.Fit(tec_degrees, discrete=True, verbose=False)

kmin = tec_fit.xmin
alpha = tec_fit.alpha
D = tec_fit.D

print(f"Estimated kmin: {kmin}")
print(f"Estimated alpha: {alpha}")
print(f"Kolmogorov-Smirnov D-statistic: {D}")

p_value = goodness_of_fit_multiprocess(tec_degrees, tec_fit, 1000)
print(f'TEC network has PVALUE: {p_value}')

alternatives = ['exponential', 'lognormal_positive', 'truncated_power_law'] # alternatives to test
for alternative in alternatives:
    curr_r, curr_p = tec_fit.distribution_compare('power_law', alternative, nested=False)
    print(f"ratio, p-value for {alternative}: ({curr_r}, {curr_p})")

Estimated kmin: 8.0
Estimated alpha: 1.8658538710650574
Kolmogorov-Smirnov D-statistic: 0.06581487793566387
TEC network has PVALUE: 0.0
ratio, p-value for exponential: (23.651634042376966, 0.30483748246156417)
ratio, p-value for lognormal_positive: (-71.93230492742833, 2.1233147924720665e-15)


Assuming nested distributions


ratio, p-value for truncated_power_law: (-97.99961349442597, 0.0)


In [8]:
ba_degrees = [G_barabasi_albert.degree(n) for n in G_barabasi_albert.nodes()]
ba_fit = powerlaw.Fit(ba_degrees, discrete=True, verbose=False)

kmin = ba_fit.xmin
alpha = ba_fit.alpha
D = ba_fit.D

print(f"Estimated kmin: {kmin}")
print(f"Estimated alpha: {alpha}")
print(f"Kolmogorov-Smirnov D-statistic: {D}")

p_value = goodness_of_fit_multiprocess(ba_degrees, ba_fit, 1000)
print(f'BA Model has PVALUE: {p_value}')

alternatives = ['exponential', 'lognormal_positive', 'truncated_power_law'] # stretched_exponential, truncated_power_law

for alternative in alternatives:
    curr_r, curr_p = ba_fit.distribution_compare('power_law', alternative, nested=False)
    print(f"ratio, p-value for {alternative}: ({curr_r}, {curr_p})")

Estimated kmin: 9.0
Estimated alpha: 2.8711151451723693
Kolmogorov-Smirnov D-statistic: 0.009889285736095044
BA Model has PVALUE: 0.642
ratio, p-value for exponential: (427.4636032021505, 1.4542420643114792e-15)
ratio, p-value for lognormal_positive: (20.657757393463932, 0.0026337047434299315)


Assuming nested distributions


ratio, p-value for truncated_power_law: (-0.5516618250334817, 0.29353785740169425)


#### Scale-free analysis by threhold

In [9]:
#! Running takes multiple hours to finish
# thresholds = [0.9, 0.85, 0.8, 0.75, 0.7, 0.65, 0.6, 0.55, 0.5]
thresholds = [0.9, 0.85]
result = []
for thresh in thresholds:
    curr_result = []
    curr_al = ne.threshold_weighted_adjacency_list(np_tec_abs, thresh)
    curr_graph = ne.construct_network(curr_al, "TEC", tec.columns)

    n = nx.number_of_nodes(curr_graph)
    ba_m = nx.number_of_edges(curr_graph) // nx.number_of_nodes(curr_graph)
    G_barabasi_albert = nx.barabasi_albert_graph(n, ba_m)

    tec_degrees = [curr_graph.degree(n) for n in curr_graph.nodes()]
    tec_fit = powerlaw.Fit(tec_degrees, discrete=True, verbose=False)

    curr_result.append(tec_fit.xmin)
    curr_result.append(tec_fit.alpha)
    curr_result.append(tec_fit.D)

    tec_degrees = [curr_graph.degree(n) for n in curr_graph.nodes()]
    pfit = goodness_of_fit_multiprocess(tec_degrees, tec_fit, 1000).item()
    curr_result.append(pfit)

    alternatives = ['exponential', 'lognormal_positive', 'truncated_power_law']
    for alternative in alternatives:
        curr_r, curr_p = tec_fit.distribution_compare('power_law', alternative, nested=False)
        curr_result.append((curr_r, curr_p))
    result.append(curr_result)

Assuming nested distributions


Assuming nested distributions


In [10]:
result_df = pd.DataFrame(
    result, 
    columns=['k_min', 'alpha', 'D statistic', 'p_fit', 'exponential', 'lognormal_positive', 'truncated_power_law']
)
result_df.insert(0, 'Threshold', thresholds)
result_df.head()

,Threshold,k_min,alpha,D statistic,p_fit,exponential,lognormal_positive,truncated_power_law
0,0.90,1.0,1.401101,0.144779,0.0,"(49.89887919185541, 0.0035566803765326345)","(-12.106117160038437, 3.7085812827541936e-05)","(-24.399915000442277, 2.8346214264729497e-12)"
1,0.85,1.0,1.509413,0.077487,0.0,"(381.17397437571276, 1.88940594226276e-48)","(-11.443810416206889, 0.022920229058544135)","(-29.403556729171598, 1.7430501486614958e-14)"


In [11]:
# rounding the results to 4 decimal points can cause small values to become 0!
rounded_result = []
for r in result:
    curr_round = []
    for curr in r:
        if type(curr) is tuple:
            curr = (round(curr[0], 4), round(curr[1], 4))
            curr_round.append(curr)
        else:
            curr_round.append(round(curr, 4))
    rounded_result.append(curr_round)

result_df = pd.DataFrame(
    rounded_result, 
    columns=['k_min', 'alpha', 'D statistic', 'p_fit', 'exponential', 'lognormal_positive', 'truncated_power_law']
)
result_df.insert(0, 'Threshold', thresholds)
result_df.head()

,Threshold,k_min,alpha,D statistic,p_fit,exponential,lognormal_positive,truncated_power_law
0,0.90,1.0,1.4011,0.1448,0.0,"(49.8989, 0.0036)","(-12.1061, 0.0)","(-24.3999, 0.0)"
1,0.85,1.0,1.5094,0.0775,0.0,"(381.174, 0.0)","(-11.4438, 0.0229)","(-29.4036, 0.0)"
